In [1]:
# Cell A: Install wapiti3
!pip install --upgrade pip
!pip install wapiti3

  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)


ERROR: To modify pip, please run the following command:
C:\Users\awans\miniconda3\envs\jupyterenv\python.exe -m pip install --upgrade pip


In [2]:
# Cell B: Configure target (replace with your Gruyere instance or other authorized target)
TARGET = "https://google-gruyere.appspot.com/418781713666967352843502231835610154724/"   
OUT_NAME = "wapiti_report.html"  # name of output HTML file (Wapiti -o)
SCAN_OPTIONS = ["-p", "5"]  # extra CLI options if desired, e.g., ["-p", "5"] (risk/level) or module filters
print("Target:", TARGET)


Target: https://google-gruyere.appspot.com/418781713666967352843502231835610154724/


In [3]:
# Cell C: Run Wapiti using subprocess with fallbacks. This runs Wapiti CLI and writes an HTML report.
import subprocess
import sys
import shlex
import os

def run_wapiti(target, out_name, extra_opts=None, timeout=None):
    extra_opts = extra_opts or []
    # Try likely command names in order
    candidates = ["wapiti", "wapiti3", f"{sys.executable} -m wapiti"]  # last tries running as module
    for cmd in candidates:
        # build command list safely
        if cmd.endswith("wapiti"):
            cmd_list = shlex.split(cmd)
        else:
            cmd_list = [cmd] if isinstance(cmd, str) else cmd
        # Append args for wapiti: -u <url> -o <outfile>
        full = cmd_list + ["-u", target, "-o", out_name] + extra_opts
        try:
            print("Trying command:", " ".join(full))
            # use shell=False where possible
            proc = subprocess.run(full, check=True, capture_output=True, text=True, timeout=timeout)
            print(proc.stdout)
            print("Wapiti finished successfully using:", cmd)
            return True
        except subprocess.CalledProcessError as e:
            print(f"Command failed ({cmd}): returncode={e.returncode}")
            print("stderr:", e.stderr[:1000])
            # try next candidate
        except FileNotFoundError:
            print(f"Command not found in PATH: {cmd}")
        except Exception as e:
            print("Error running command:", e)
    return False

# run the scan (warning: can take time depending on target and options)
success = run_wapiti(TARGET, OUT_NAME, extra_opts=SCAN_OPTIONS, timeout=600)
print("Scan succeeded?" , success)


Trying command: wapiti -u https://google-gruyere.appspot.com/418781713666967352843502231835610154724/ -o wapiti_report.html -p 5
Command failed (wapiti): returncode=1
stderr: (trapped) error reading bcrypt version
Traceback (most recent call last):
  File "C:\Users\awans\miniconda3\envs\jupyterenv\Lib\site-packages\passlib\handlers\bcrypt.py", line 620, in _load_backend_mixin
    version = _bcrypt.__about__.__version__
              ^^^^^^^^^^^^^^^^^
AttributeError: module 'bcrypt' has no attribute '__about__'
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\awans\miniconda3\envs\jupyterenv\Scripts\wapiti.exe\__main__.py", line 4, in <module>
  File "C:\Users\awans\miniconda3\envs\jupyterenv\Lib\site-packages\wapitiCore\main\wapiti.py", line 33, in <module>
    from wapitiCore.controller.wapiti import InvalidOptionValue, module_to_class_name, Wapiti
  File "C:\Users\awans\minicon

In [4]:
# Cell D: Find the generated HTML and render it inline in the notebook
import os, glob
from IPython.display import display, HTML

def find_html_report(out):
    # If out is a file and ends with .html, use it
    if os.path.isfile(out) and out.lower().endswith('.html'):
        return os.path.abspath(out)
    # If out is a directory, search for newest .html inside it
    if os.path.isdir(out):
        matches = glob.glob(os.path.join(out, "*.html"))
        if matches:
            matches.sort(key=os.path.getmtime, reverse=True)
            return os.path.abspath(matches[0])
    # Search common patterns
    candidates = glob.glob(out + "*/*.html") + glob.glob(out + "*.html") + glob.glob("*.html")
    if candidates:
        candidates.sort(key=os.path.getmtime, reverse=True)
        return os.path.abspath(candidates[0])
    return None

report_path = find_html_report(OUT_NAME)
if report_path:
    print("Report file found:", report_path)
    try:
        with open(report_path, 'r', encoding='utf-8') as f:
            html = f.read()
        display(HTML(html))
    except Exception as e:
        print("Could not render HTML inline (error):", e)
        print("Open the file in your browser instead:", report_path)
else:
    print("No HTML report found. Check that the scan produced the report and that OUT_NAME matches the -o argument passed to Wapiti.")


No HTML report found. Check that the scan produced the report and that OUT_NAME matches the -o argument passed to Wapiti.
